In [7]:
import cv2 
import os 
import glob
import json
import csv
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd

from tqdm import tqdm 
from module.utils import imread, imfindcircles
from module.disp_measure import displacement_measure

In [2]:
# 센서 번호 입력
sensor_num = 196
sensor_num = str(sensor_num)

# 센서 번호에 따른 이미지 리스트 호출
img_list = glob.glob(os.path.join(r'imgs', sensor_num, '*.jpg'))

# 센서 번호에 따른 세부 파라메터 호출 
with open('params.json') as param_config_json : 
    param_config = json.load(param_config_json)
    
params = param_config[sensor_num]


result = [] # 결과를 저장할 result list 

# 최초 source image에서 circle detection 수행 
src_img = imread(img_list[0])
centers, r_estimated, metric = imfindcircles(src_img, 
                                             [params['min_rad'], params['max_rad']],
                                            sensitivity = 0.96)
src_circles = np.concatenate((centers, r_estimated[:,np.newaxis]), axis = 0).T
src_circles = np.squeeze(src_circles)

# 호출된 이미지 목록에서 displacement_measure 수행 
for img_path in tqdm(img_list[:100]): 
    dest_img = imread(img_path)
    
    '''
    변경사항
    1. 함수 이름을 displacement_measure로 변경하였습니다. 
    2. displacement_measure에 필요한 변수는 아래와 같습니다. 
        1) dest_img : 측정 대상 이미지 
        2) src_img : 고정점 이미지 
        3) src_circles : 고정점 이미지에서 탐지된 원들의 중심 좌표 및 지름 (고정점 이미지에서 변위 재측정 방지용)
        4) min_rad : 최소 원의 지름 크기 / max_rad : 최대 원의 지름 크기 
    '''
    disp, dest_circles = displacement_measure(dest_img,
                                              src_img,
                                              src_circles = src_circles,                                            
                                              min_rad = params['min_rad'],
                                              max_rad = params['max_rad'])
    
    result.append(disp)
    for dest_circle in dest_circles : 
        dest_img = cv2.circle(dest_img, tuple(np.asarray(dest_circle[:2], dtype =np.int64)), int(dest_circle[2]), (0,255,0), thickness=1, lineType=8, shift=0)
    cv2.imwrite(img_path[:-4] + '_with_circles_result.jpeg', dest_img)
    
collect_rslt = []
for rslt in result : 
    collect_rslt.append(rslt[0])
    
plt.plot(np.array(collect_rslt))

IndexError: list index out of range

In [2]:
## check the circle detection accuracy 

param1 = 200
param2 = 14
min_rad = 75
max_rad = 90
img = imread(r'imgs\196\Img_196_20200829_190100.jpg') 


centers, r_estimated, metric = imfindcircles(img, 
                                             [min_rad, max_rad],
                                             sensitivity = 0.99)
circles = np.concatenate((centers, r_estimated[:,np.newaxis]), axis = 0).T
circles = np.squeeze(circles)

# Draw circles that are detected. 
if circles is not None: 
  
    # Convert the circle parameters a, b and r to integers. 
    circles = np.uint16(np.around(circles)) 
  
    for pt in circles: 
        a, b, r = pt[0], pt[1], pt[2] 
  
        # Draw the circumference of the circle. 
        cv2.circle(img, (a, b), r, (0, 255, 0), 2) 
  
        # Draw a small circle (of radius 1) to show the center. 
        cv2.circle(img, (a, b), 1, (0, 0, 255), 3) 
        cv2.imshow("Detected Circle", img) 
        cv2.waitKey(0) 
else:
        cv2.imshow('Circle is not detected', img)
        cv2.waitKey(0) 


In [13]:
start_p = time.time()

sensitivity = 0.96
centers = []
r_estimated = []
metric = []
radiusRange = [35, 45]

A = imread(img_list[0]) 

centers, r_estimated, metric = imfindcircles(A, radiusRange,  ObjectPolarity = 'bright', sensitivity = 0.95)


In [18]:
img_list = glob.glob(os.path.join(r'data', '*.jpg'))

sensor_num_list = []
for img_name in img_list:
    sensor_num_list.append(img_name[9:12])
    
sensor_list = np.unique(sensor_num_list)

for sensor_name in sensor_list: 
    sensor_img_list = glob.glob(os.path.join(r'data', '*' + sensor_name + '*.jpg'))
    init_img = cv2.imread(sensor_img_list[0])
    cv2.imwrite(os.path.join('data', 'Img_' + sensor_name + '_01_center.jpg'),  init_img)

In [8]:
img_list = glob.glob(os.path.join('/media/sss/Seagate Backup Plus Drive/data', 'Img*.jpg'))

result_list = []
sensitivity = 0.98

# 센서 번호에 따른 세부 파라메터 호출 
with open('params.json') as param_config_json : 
    param_config = json.load(param_config_json)


for num, img_file in tqdm(enumerate(img_list)) : 
    
    sensor_num = str(os.path.basename(img_file)[4:7])
    if sensor_num in param_config: 
        print(os.path.basename(img_file))
        params = param_config[sensor_num]
        img = imread(img_file)
        centers, r_estimated, metric = imfindcircles(img, 
                                             [params['min_rad'], params['max_rad']],
                                            sensitivity = sensitivity)
        circles = np.concatenate((centers, r_estimated[:,np.newaxis]), axis = 0).T
        circles = np.squeeze(circles)
        
        result_list.append([os.path.basename(img_file),circles, sensitivity])
        
        if num%1000 == 0:
            df = pd.DataFrame(result_list)
            df.to_csv('displacement_measurement_047_059.csv',index=False)

0it [00:00, ?it/s]

Img_047_20200622_140100.jpg


1it [00:04,  4.04s/it]

Img_047_20200622_150100.jpg


2it [00:07,  3.95s/it]

...

38241it [7:01:30,  4.78s/it]

Img_056_20200624_080100.jpg


38242it [7:01:34,  4.56s/it]

Img_056_20200624_090100.jpg


38243it [7:01:39,  1.51it/s]


In [ ]:
# array(['068', '071', '074', '077', '090'], dtype='<U3')


# 센서 번호에 따른 세부 파라메터 호출 
with open('params.json') as param_config_json : 
    param_config = json.load(param_config_json)

img_list = glob.glob(os.path.join('/media/sss/Seagate Backup Plus Drive/data', '*.jpg'))

# img = imread(img_list[100])
# plt.imshow(img)

img_num_list = []
for img_filepath in img_list:
    img_num_list.append(os.path.basename(img_filepath)[4:7])

plt.figure(figsize=(20, 20))
plt.imshow(imread(img_list[0]))
# np.unique(img_num_list)
img_list[0]